In [1]:
import re
import pandas as pd


In [2]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer

# Загрузка ресурсов для лемматизатора
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alex4\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
test_data = pd.read_parquet("C:/Users/alex4/Desktop/code/data/test.parquet")
train_data = pd.read_parquet("C:/Users/alex4/Desktop/code/data/train.parquet")

In [5]:

# Email
def match_emails(text):
    pattern = r'\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b'
    return re.findall(pattern, text)

# Телефоны
def match_phones(text):
    pattern = r'\b(?:\+7|8)[-\s\(]?\d{3}[\)\s-]?\d{3}[-]?\d{2}[-]?\d{2}\b'
    return re.findall(pattern, text)

# Адреса
def match_addresses(text):
    pattern = r'г\.\s?[А-Яа-я\-]+,?\s+ул\.?\s?[А-Яа-я\- ]+,?\s+д\.?\s?\d+,?\s+кв\.?\s?\d+'
    return re.findall(pattern, text)

# Эмотиконы
def match_emoticons(text):
    pattern = r'[:;=8][-~]?[)D\(]'
    return re.findall(pattern, text)

# Формулы
def match_math(text):
    pattern = r'[\w\s]*=\s*[\w\s\*\+\-/\(\)\^=]+'
    return re.findall(pattern, text)

# Сокращения
def match_abbreviations(text):
    pattern = r'\b(?:[A-ZА-Я][a-zа-я]{0,3}\.)\s?[A-ZА-Я][a-zа-я]+\b'
    return re.findall(pattern, text)


In [10]:
def test_all():
    print("Emails:", match_emails("Мой адрес: test.user@mail.ru и ещё один: abc@domain.com"))
    print("Phones:", match_phones("Позвони мне +7-901-000-00-00 или 8(918)3213412"))
    print("Addresses:", match_addresses("г. Санкт-Петербург, ул. Советская, д. 1294124, кв. 1"))
    print("Emoticons:", match_emoticons("Привет :) как дела? Всё норм :D"))
    print("Math:", match_math("Уравнение: a = b*c = (c+d)^2 и ещё x = y+z"))
    print("Abbreviations:", match_abbreviations("Dr. Ivanov — известный специалист. Prof. Petrov тоже"))


In [11]:
test_all()

Emails: ['test.user@mail.ru', 'abc@domain.com']
Phones: ['8(918)3213412']
Addresses: ['г. Санкт-Петербург, ул. Советская, д. 1294124, кв. 1']
Emoticons: [':)', ':D']
Math: [' a = b*c = (c+d)^2 и ещё x = y+z']
Abbreviations: ['Dr. Ivanov', 'Prof. Petrov']


In [6]:

def segment_and_tokenize_custom(text: str):
    # Список сокращений, которые не должны разделяться
    abbreviations = ['Dr.', 'Mr.', 'Ms.', 'Mrs.', 'Prof.', 'г.', 'ул.', 'д.', 'кв.', 'st.']

    # Экранируем сокращения
    for abbr in abbreviations:
        text = text.replace(abbr, abbr.replace('.', '<DOT>'))

    # Разделение на предложения по [.!?] + пробел + заглавная буква
    sentence_split_pattern = re.compile(r'([.!?])\s+(?=[А-ЯA-Z])')
    text = sentence_split_pattern.sub(r'\1<<<SENT>>>', text)

    # Возвращаем точки обратно в сокращениях
    text = text.replace('<DOT>', '.')

    # Разделяем на предложения
    sentences = text.split('<<<SENT>>>')

    # Общий токенайзер
    token_pattern = re.compile(r"""
        \b(?:\+7|8)[-\s(]?\d{3}[-\s)]?\d{3}[-]?\d{2}[-]?\d{2}\b |         # телефоны
        \b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b |              # email
        г\.\s?[А-Яа-я\-]+,?\s+ул\.?\s?[А-Яа-я\- ]+,?\s+д\.?\s?\d+,?\s+кв\.?\s?\d+ |  # адреса
        [:;=8][-~]?[)D\(] |                                               # эмотиконы
        (?:[A-ZА-Я][a-zа-я]{0,3}\.)\s?[A-ZА-Я][a-zа-я]+ |                 # сокращения + фамилия
        \b\w+\s*=\s*[\w\s\*\+\-/\(\)\^=]+ |                               # формулы
        [А-Яа-яA-Za-z0-9]+(?:-[А-Яа-яA-Za-z0-9]+)* |                     # слова
        [.,!?;:] |                                                       # пунктуация
        [^\s]                                                            # прочее
    """, re.VERBOSE)

    tokenized_sentences = []
    for sentence in sentences:
        tokens = token_pattern.findall(sentence.strip())
        if tokens:
            tokenized_sentences.append(tokens)

    return tokenized_sentences


In [8]:
if __name__ == "__main__":
    text = (
        "Dr. Ivanov отправил-письмо на адрес abc@abc.abc. "
        "Позвони по номеру +7-901-000-00-00. "
        "Привет:), как дела? "
        "Формула: a = b*c = (c+d)^2"
    )

    result = segment_and_tokenize_custom(text)
    for i, sent in enumerate(result):
        print(f"Предложение {i+1}: {sent}")


Предложение 1: ['Dr. Ivanov', 'отправил-письмо', 'на', 'адрес', 'abc@abc.abc', '.']
Предложение 2: ['Позвони', 'по', 'номеру', '+', '7-901-000-00-00', '.']
Предложение 3: ['Привет', ':)', ',', 'как', 'дела', '?']
Предложение 4: ['Формула', ':', 'a = b*c = (c+d)^2']


In [9]:
def stem_tokens(tokens, language='english'):
    """
    Выполняет стемминг списка токенов.
    """
    stemmer = SnowballStemmer(language)
    return [stemmer.stem(token) for token in tokens]


In [10]:
def lemmatize_tokens(tokens):
    """
    Выполняет лемматизацию списка токенов на английском языке.
    (Для русского лучше использовать pymorphy2 — скажешь, добавим.)
    """
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]


In [11]:
if __name__ == "__main__":
    sentence = ['refredgirator','generations']

    stems = stem_tokens(sentence)
    lems = lemmatize_tokens(sentence)

    print("Токены:       ", sentence)
    print("Стеммы:       ", stems)
    print("Леммы:        ", lems)


Токены:        ['refredgirator', 'generations']
Стеммы:        ['refredgir', 'generat']
Леммы:         ['refredgirator', 'generation']


In [21]:
# Функция для формирования аннотации в формате .tsv
def generate_tsv_annotation(text: str):
    sentences = segment_and_tokenize_custom(text)
    
    result = []
    for idx, sentence in enumerate(sentences):
        # Токенизация, стемминг и лемматизация для каждого предложения
        stems = stem_tokens(sentence, language='english')
        lems = lemmatize_tokens(sentence)

        for token, stem, lemma in zip(sentence, stems, lems):
            result.append(f"{token}\t{stem}\t{lemma}")
        
        # Добавление пустой строки между предложениями
        if idx < len(sentences) - 1:
            result.append("")
    
    # Сохранение в файл
    with open("output_annotation.tsv", "w", encoding="utf-8") as f:
        f.write("\n".join(result))




In [ ]:
def generate_tsv_annotation_from_df(df, output_path: str, language='english'):
    result = []
    В
    for index, row in df.iterrows():
        text = row['text']
        sentences = segment_and_tokenize_custom(text)
        
        for idx, sentence in enumerate(sentences):
            if language == 'english':
                stems = stem_tokens(sentence, language='english')
                lems = lemmatize_tokens(sentence)
            for token, stem, lemma in zip(sentence, stems, lems):
                result.append(f"{token}\t{stem}\t{lemma}")
            

    # Сохранение в файл
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(result))


In [39]:
# Пример использования
generate_tsv_annotation_from_df(train_data, "train.tsv", language="english")


In [29]:
test_data

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [12]:
def generate_tsv_annotation_from_df1(df, output_path: str, language='english'):
    result = []

    # Оставляем только строки с label == 0
    filtered_df = df[df['label'] == 0]

    for index, row in filtered_df.iterrows():
        text = row['text']
        sentences = segment_and_tokenize_custom(text)

        for idx, sentence in enumerate(sentences):
            if language == 'english':
                stems = stem_tokens(sentence, language='english')
                lems = lemmatize_tokens(sentence)

            for token, stem, lemma in zip(sentence, stems, lems):
                result.append(f"{token}\t{stem}\t{lemma}")

    # Сохраняем результат в файл
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(result))


In [18]:
generate_tsv_annotation_from_df1(test_data, "output_label1.tsv")


In [44]:
import os
import pandas as pd
from pathlib import Path

def generate_tsv_annotations_by_label(df, output_dir: str, language='english'):
    """
    Генерирует TSV-файлы с аннотациями, распределяя их по папкам в соответствии с метками
    
    Параметры:
        df: DataFrame с колонками 'text' и 'label'
        output_dir: корневая директория для сохранения файлов
        language: язык текста ('english' по умолчанию)
    """
    # Создаем корневую директорию, если ее нет
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Группируем данные по меткам
    grouped = df.groupby('label')
    
    for label, group in grouped:
        # Создаем поддиректорию для метки
        label_dir = os.path.join(output_dir, str(label))
        Path(label_dir).mkdir(parents=True, exist_ok=True)
        
        # Формируем имя файла
        output_path = os.path.join(label_dir, f"annotations_{label}.tsv")
        
        result = []
        
        for index, row in group.iterrows():
            text = row['text']
            sentences = segment_and_tokenize_custom(text)
            
            for sentence in sentences:
                if language == 'english':
                    stems = stem_tokens(sentence, language='english')
                    lems = lemmatize_tokens(sentence)
                
                for token, stem, lemma in zip(sentence, stems, lems):
                    result.append(f"{token}\t{stem}\t{lemma}")

                result.append("")

        # Сохранение в файл
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("\n".join(result))
            
        print(f"Saved annotations for label '{label}' to {output_path}")

In [48]:
# Предположим, df - ваш DataFrame с колонками 'text' и 'label'
generate_tsv_annotations_by_label(train_data, "output_annotations", language='english')

Saved annotations for label '0' to output_annotations\0\annotations_0.tsv
Saved annotations for label '1' to output_annotations\1\annotations_1.tsv
Saved annotations for label '2' to output_annotations\2\annotations_2.tsv
Saved annotations for label '3' to output_annotations\3\annotations_3.tsv


In [ ]:
test_data[test_data['label'] == 0]


In [ ]:

import pandas as pd

# Устанавливаем максимальное количество строк для вывода
pd.set_option('display.max_rows', None)  # Устанавливает, чтобы показывались все строки

# Выводим данные
print(filtered_data)
